## Introduction

In this problem, you're going to write a function that will parse (a simplified version of) XML files into a Python object. Although it's not recommended that you use the parser you construct for anything serious (many excellent Python libraries already exist for parsing XML, such as the lxml library), XML files represent a fairly complex file format, that necessitates using regular expressions and recursion (or a stack) to parse these in a reasonably efficient manner.  So while it's not likely that you will need to write you own XML parser, chances are if/when you _do_ need to write a parser for some format for which there exists no good Python library, the techniques you use here will be useful for writing this parser as well.

## XML

XML stands for the eXtensible Markup Language.  It appeared as a successor to SGML (standard generalized markup language) and HTML (hypertext markup language, the standard for displaying web pages), but with some additional structure that makes the documents more well-defined; for instance, in HTML it's common for open tags to appear without a corresponding close tag, which is not allowed in pure XML.

You may already be familiar with XML, but if not the official resource for learning about the format is here https://www.w3.org/XML/ and a good resource with some concrete examples is here http://www.w3schools.com/xml/.  We'll assume here that you're broadly familiar with the basic ideas behind XML, and just describe what you need to know to complete the parser for this assignment.

Here is an example XML document:
    
    <?xml version="1.0" encoding="UTF-8"?>
    <!-- This is a comment -->
    <note date="8/31/12">
        <to>Tove</to>
        <from>Jani</from>
        <heading type="Reminder"/>
        <body>Don't forget me this weekend!</body>
        <!-- This is a multiline comment,
             which take a bit of care to parse -->
    </note>
    
There are a few elements here of importance.
1. Tags are denoted `<tag_name>content</tag_name>` where `<tag_name>` is the opening tag and `</tag_name>` is the closing tag.  All text (including whitespace) between these tags is the content.
2. Attributes follow a tag, and are written as a list of `attr_name="attribute_value"` pairs, where we can use either double quotes or single quotes around the attribute value.  If you use double quotes then a single quote can appear in the text and vice versa.  There can be whitespace around the equals sign or not.
3. If a tag has no content `<tag_name attr_name="attribute_value"></tag_name>` can be abbreviated as the open/close tag `<tag_name attr_name="attribute_value"/>`. In some cases, such as in HTML5, you might come across tags that have no content but aren't closed, such as `<meta ...>` and `<link ...>` tags. **However, as we are dealing with XML (a stricter context), tags are required to be closed.** If you're interested, [this document](https://www.w3schools.com/html/html_xhtml.asp) on XHTML vs HTML touches upon this idea.
4. A XML prologue is written as `<?tag_name attr_name="attribute_value"?>`.  It has no close tag.  We'll also consider documents that allow for an HTML declaration, such as `<!DOCTYPE html>` (this will let us parse some HTML documents that are well-formed enough to also parse as valid XML).
5. Comments are denoted by `<!-- comment_text -->`  and the comment text can span multiple lines.

## Q1: Regular expression for identifying tags

First, we'll use regular expressions to identify tags and other elements of XML files.  Specifically, you'll need to create 6 regular expressions that locate open tags, close tags, open/close tags, comments, xml_prolog, and html declarations.  For the open, close and open/close tags, make sure that your regular expression also matches and returns 1) the tag name, and 2) all the attributes.  Note that in our implementation, we actually have the open tag _also_ match open/close tags, but you are free to do this either way (they can match or not).  Comments may be split across multiple lines, but you can assume that all other tags must occur on a single line (without newlines within the tag itself).

In [1]:
#### Q1: FILL IN THIS CELL
import re
tag_open = re.compile("<([\w]+)[ ]*([^>]*)")
tag_close = re.compile("</([\w]*)[>]()")
tag_open_close = re.compile("<([\w]+)[ ]*([^>]*)/>")

comment = re.compile("<!--(.*?)-->",flags=re.DOTALL)
xml_prolog = re.compile("<\?([\w\W]*)\?>")
html_declaration = re.compile("<!([\w\s[\]]+[-]*)>")

You can test your code on the following snippet, and on the HTML source of our course web page.

In [8]:
import requests
test_snippet = """<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE xml> <!-- not actually valid xml-->
<!-- This is a comment -->
<note date="8/31/12">
    <to>Tove</to>
    <from>Jani</from>
    <heading type="Reminder"/>
    <body>Don't forget me this weekend!</body>
    <!-- This is a multiline comment,
         which take a bit of care to parse -->
</note>
"""

# [NOTE] Comment this out prior to submission
course_webpage = str(requests.get("http://www.datasciencecourse.org/2016").content)


When you run the cell below (you should make a copy of them so you don't accidentally overwrite), it should produce the following output.  

Example output:
```python
tag_open:  [('note', ' date="8/31/12"'), ('to', ''), ('from', ''), ('heading', ' type="Reminder"/'), ('body', '')]
tag_close:  [('to', ''), ('from', ''), ('body', ''), ('note', '')]
tag_open_close:  [('heading', ' type="Reminder"')]
comment:  [' not actually valid xml', ' This is a comment ', ' This is a multiline comment,\n         which take a bit of care to parse ']
xml_prolog:  ['xml version="1.0" encoding="UTF-8"']
html_declaration:  ['DOCTYPE xml']
```

In [28]:
print("tag_open: ", tag_open.findall(test_snippet))
print("tag_close: ", tag_close.findall(test_snippet))
print("tag_open_close: ", tag_open_close.findall(test_snippet))
print("comment: ", comment.findall(test_snippet))
print("xml_prolog: ", xml_prolog.findall(test_snippet))
print("html_declaration: ", html_declaration.findall(test_snippet))

tag_open:  [('note', 'date="8/31/12"'), ('to', ''), ('from', ''), ('heading', 'type="Reminder"/'), ('body', '')]
tag_close:  [('to', ''), ('from', ''), ('body', ''), ('note', '')]
tag_open_close:  [('heading', 'type="Reminder"')]
comment:  [' not actually valid xml', ' This is a comment ', ' This is a multiline comment,\n         which take a bit of care to parse ']
xml_prolog:  ['xml version="1.0" encoding="UTF-8"']
html_declaration:  ['DOCTYPE xml']


Similarly, the cell below should produce the following counts. 

Example output:
```python
tag_open:  469
tag_close:  439
tag_open_close:  30
comment:  23
xml_prolog:  0
html_declaration:  2
```

In [1]:
# [NOTE] Comment this out prior to submission
# print("tag_open: ", len(tag_open.findall(course_webpage)))
# print("tag_close: ", len(tag_close.findall(course_webpage)))
# print("tag_open_close: ", len(tag_open_close.findall(course_webpage)))
# print("comment: ", len(comment.findall(course_webpage)))
# print("xml_prolog: ", len(xml_prolog.findall(course_webpage)))
# print("html_declaration: ", len(html_declaration.findall(course_webpage)))

(Note that although there is only one html declaration in the course webpage, there is a field _within_ a comment that looks suspiciously like a declaration, so you can pick up both of these).

## Q2: XML Parser

Using the regular expressions above, now you'll write an XML parser (although technically you don't _have_ to use them, you could try to write a complete XML parser using a single regular extended expression if you really want to, but we would highly advise against this).  Specifically, you should fill in the `__init__` function for the class prototype below.

In [3]:
#### Q2: fill in this cell
class XMLNode:
    
    def __init__(self, tag, attributes, content):
        self.tag = tag
        self.attributes = attributes
        self.children =  []
        self.content = content
        
        self.startposn=0
        self.endposn=0
        
    #def parsing(self):
        tag_open = re.compile("<([\w]+)[ ]*([^>]*)")
        tag_close = re.compile("</([\w]*)[>]()")
        tag_open_close = re.compile("<([\w]+)[ ]*([^>]*)/>")
        comment = re.compile("<!--(.*?)-->",flags=re.DOTALL)
        xml_prolog = re.compile("<\?([\w\W]*)\?>")
        html_declaration = re.compile("<!([\w\s[\]]+[-]*)>")
        attributes = re.compile(r"[ ]*([\w-]+)[=]{1}[\"|']([^\"]*)")
                
        
        
        while(self.startposn<(len(content)-1)):
            try:
                self.startposn = content.index("<",self.startposn)
            except:
                return

            strt= tag_open.match(content,self.startposn)
            close= tag_close.match(content,self.startposn)
            openclose= tag_open_close.match(content,self.startposn)
            cmmnt= comment.match(content,self.startposn)
            xmls= xml_prolog.match(content,self.startposn)
            html= html_declaration.match(content,self.startposn)
            
            if(xmls!=None):
                self.startposn=xmls.end()
                continue
            elif(html!=None):
                self.startposn=html.end()
                continue
            elif(cmmnt!=None):
                self.startposn=cmmnt.end()
                continue
            elif(strt!=None):
                if(openclose!=None):
                    attributedict={}
                    if(openclose.group(2)!=""):
                        dictmatch = attributes.finditer(openclose.group(2))
                        for matches in dictmatch:
                            attributedict[matches.group(1)]= matches.group(2)
                        
                    self.children.append(XMLNode(openclose.group(1),attributedict,"")) 
                    self.startposn= openclose.end()
                    continue
                else:
                    attributedict={}
                    if(strt.group(2)!=""):
                        dictmatch = attributes.finditer(strt.group(2))
                        for matches in dictmatch:
                            attributedict[matches.group(1)]= matches.group(2)                    
                    self.children.append(XMLNode(strt.group(1),attributedict,content[strt.end()+1:]))
                    self.startposn= strt.end() + self.children[-1].endposn
                    continue
            elif(close!=None):
                if close.group(1) != tag:
                    raise Exception("Error: <{0}> tag closed with {1}".format(tag, close.group()))
                else:
                     self.content = self.content[:close.start()]
                self.endposn = close.end()
                self.content = content[:close.start()]
                return
            
    def find(self, tag, **kwargs):
        """
        Search for a given tag and atributes anywhere in the XML tree

        Args:
            tag (string): tag to match
            kwargs (dictionary): list of attribute name / attribute value pairs to match

        Returns:
            (list): a list of XMLNode objects that match from anywhere in the tree
        """
        list_match=[]
        #Reference:https://stackoverflow.com/questions/4527942/comparing-two-dictionaries-in-python
        #Author:mouad
        if self.tag == tag and (len(set(self.attributes.items()) & set(kwargs.items()))==len(kwargs)):
            list_match.append(self)
        elif self.tag == tag and len(kwargs)==0:
            list_match.append(self)
            
            
        if len(self.children)>0:
            for allobj in self.children:
                list_match.extend(allobj.find(tag,**kwargs))
                continue
        else:
            return list_match 
        return list_match
    

There is a lot that this function needs to do, which is best explained by an example.  We'll eventually parse the `test_snippet` above using the command:
    
    root = XMLNode("", {}, test_snippet)

This will create a root node with an empty tag, an empty dictionary for the attributes, and created by parsing the test_snippet; the `content` attribute here will contain the entire test snippet.

The `children` attribute is a list that contains a single XMLNode instance, corresponding to the `note` tag.  This instance is created (you don't call this function directly, rather each node must recursively create all its children, so the call above would recursively make the following call to create its child) by calling XMLNode with the parameters

    XMLNode("note", {"date":"8/31/12"}, test_snippet[133:])
    # (test_snippet[133:] just happens to be the position that immediately follows the <note> open tag)
    
This child node will have the given tag and attributes, plus content given by only the `content` equals to the string that occurs _within_ the note open and close tags.  It will similarly have four children, one corresponding to each of the subtags.  The following code illustrates how you can use XMLNode to parse test_snippet, and what the structure is after you perform the parsing.

Example output:
```python
root.tag:  
root.attributes:  {}
root.content:  '<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE xml> <!-- not actually valid xml-->\n<!-- This is a comment -->\n<note date="8/31/12">\n    <to>Tove</to>\n    <from>Jani</from>\n    <heading type="Reminder"/>\n    <body>Don\'t forget me this weekend!</body>\n    <!-- This is a multiline comment,\n         which take a bit of care to parse -->\n</note>\n'
root.children:  [<__main__.XMLNode instance at 0x10425c0e0>]

note.tag:  note
note.attributes:  {'date': '8/31/12'}
note.content:  '\n    <to>Tove</to>\n    <from>Jani</from>\n    <heading type="Reminder"/>\n    <body>Don\'t forget me this weekend!</body>\n    <!-- This is a multiline comment,\n         which take a bit of care to parse -->\n'
note.children:  [<__main__.XMLNode instance at 0x10425c098>, <__main__.XMLNode instance at 0x10425c128>, <__main__.XMLNode instance at 0x10425c368>, <__main__.XMLNode instance at 0x10425c488>]

to.tag:  to
to.attributes:  {}
to.content:  'Tove'
to.children:  []

heading.tag:  heading
heading.attributes:  {'type': 'Reminder'}
heading.content:  ''
heading.children:  []
```

In [9]:
# root = XMLNode("", {}, course_webpage)
# print("root.tag: ", root.tag)
# print("root.attributes: ", root.attributes)
# print("root.content: ", repr(root.content))
# print("root.children: ", root.children)
# print("")
# print("note.tag: ", root.children[0].tag)
# print("note.attributes: ", root.children[0].attributes)
# print("note.content: ", repr(root.children[0].content))
# print("note.children: ", root.children[0].children)
# print("")
# print("to.tag: ", root.children[0].children[0].tag)
# print("to.attributes: ", root.children[0].children[0].attributes)
# print("to.content: ", repr(root.children[0].children[0].content))
# print("to.children: ", root.children[0].children[0].children)
# print("")
# print("heading.tag: ", root.children[0].children[2].tag)
# print("heading.attributes: ", root.children[0].children[2].attributes)
# print("heading.content: ", repr(root.children[0].children[2].content))
# print("heading.children: ", root.children[0].children[2].children)


Additionally, if you pass an XML object that is poorly formated, in that there is some mismatched open and close tag, the function should raise an exception. Note: comment out this cell prior to submission.

Example output: 
```python
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
<ipython-input-68-f654067d17d4> in <module>()
----> 1 root = XMLNode("", {}, "<note><to>You</from></note>")

<ipython-input-67-8c4c856beda8> in __init__(self, tag, attributes, content)
     37                 else:
     38                     # if it's an open tag, parse it recursively
---> 39                     self.children.append(XMLNode(m.group(1), attributes, content[m.end():]))
     40                     pos = m.end() + self.children[-1].endpos
     41                 continue

<ipython-input-67-8c4c856beda8> in __init__(self, tag, attributes, content)
     37                 else:
     38                     # if it's an open tag, parse it recursively
---> 39                     self.children.append(XMLNode(m.group(1), attributes, content[m.end():]))
     40                     pos = m.end() + self.children[-1].endpos
     41                 continue

<ipython-input-67-8c4c856beda8> in __init__(self, tag, attributes, content)
     45             if m:
     46                 if m.group(1) != tag:
---> 47                     raise Exception("Error: <{0}> tag closed with {1}".format(tag, m.group()))
     48                 else:
     49                     self.content = self.content[:m.start()]

Exception: Error: <to> tag closed with </from>
```

In [4]:
# [NOTE] Comment this out prior to submission
# root = XMLNode("", {}, "<note><to>You</from></note>")

Finally, your code should also be able to parse the course webpage (which we made sure was valid XML).

Example output (the total count may vary slightly based on changes we make to the website): 
```python
>>> print total_count(root)
345
```

In [5]:
def total_count(n):
    """ Gets the total number of nodes in an XMLNode tree. """
    return len(n.children) + sum(total_count(c) for c in n.children)
# [NOTE] Comment this out prior to submission
# root = XMLNode("", {}, test_snippet)
# print (total_count(root))


5


Lets discuss in a bit more detail how the XML parsing will work algorithmically.  You being the initializer by copying the provided parameters to the class attributes.  Note that if you want you could make a full string copy here, but we don't bother.  Now we begin parsing the file, which we do by repeating the following logic until termination:
1. Look for the next xml tag (or comment, etc), in the file.  This is best done by finding the next `'<'` character.  If you can't find any, return.
2. If it's an xml prolog, html declaration, or comment, ignore this portion, and continue parsing after the prolog, declaration, or comment (i.e., throw away whatever information is contained in these portions)
3. If it's an open tag, read its tag and attributes (you'll likely want to use a regular expression to parse the attributes as well, but we leave this up to you).  If it's just an open tag, then recursively create an XMLNode object initializer this tag and attributes, and the content that occurred after the open tag.  If it's an open/close tag, create a XMLNode the same as before but with empty content.
4. If it's a close tag, make sure that the close tag matches the tag originally provided to the current XMLNode constructor (otherwise, we have a situation where one tag is closed with a different tag), and raise an Exception if not.  If the tags do match, then truncate the content to contain only the content before the closed tag matched, and return.

Some hints that we believe will be helpful:
1. Keep track of the current position where you are parsing the file, and make sure to properly increment this so you move past any tag that you have parsed.
2. Make use of the `match = regular_expression_obj.match(string, pos)` function, which looks for a match to the regular expression starting _exactly_ as position `pos` in `string`.  If this function returns `None`, then the regular expression did not match.  In the returned `match` object, `match.end()` contains the position where the match ended.

## Q3: Searching for tags

One of the nicer elements of the `BeautifulSoup` library is the ability to quickly search for tags that have certain attributes, without worrying about the specific structure of the model (i.e., how many levels deep the tag is, how many may exist in the document etc).  We're going to implement a similar function in our `XMLNode` class, specifically a function of the following form.

In [6]:
#### Q3: fill out this function and move it into the XMLNode class above. [NOTE] Do not leave this here for your final submission. 
            



For those who haven't seen the `**kwargs` parameter before, this is just a way to pass a variable-length list of parameters to a Python function as function parameters.  For example, you could call `find` via

    root.find("link", rel="stylesheet")
    
and in the `find` function, `kwargs` would be a dictionary equal to `{"rel":"stylesheet"}`.

This function should return a list of _all_ XMLNodes that are descendents (children, children of children, etc), of the node you call it on.  The following constains some examples of this function can be used.

Example output: 
```python
['#page-top', '#page-top', '#overview', '#schedule', '#assignments', '#instructors', '#faq', '#overview', 'intro.pdf', 'https://scs.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=3f116819-0e26-4f79-bcee-f4cb7e745773', 'http://www.zicokolter.com/', 'http://cs.cmu.edu/~ericwong/', 'https://piazza.com/class/is8ghyly8um4ew', 'https://piazza.com/class/is8ghyly8um4ew']
['8/29', '8/31', '9/7', '9/14', '9/19', '9/21', '9/26', '9/28', '10/3', '10/5', '10/10', '10/12', '10/17', '10/19', '10/24', '10/26', '10/31', '11/2', '11/7', '11/9', '11/14', '11/16', '11/21', '11/28', '11/30', '12/5', '12/7']
```

In [16]:
# [NOTE] Comment this out prior to submission
# 
# Get a list of all links on the page
# links = root.find("a")
# print([l.attributes["href"] for l in links])
# 
# Get a list of all lecture dates for the course
# tbody = root.find("section", id="schedule")[0].find("table")[0].find("tbody")[0]
# print ([a.find("td")[0].content for a in tbody.find("tr") if len(a.find("td")) > 1])

['#page-top', '#page-top', '#overview', '#schedule', '#assignments', '#instructors', '#faq', '#overview', 'intro.pdf', 'https://scs.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=3f116819-0e26-4f79-bcee-f4cb7e745773', 'data_collection.pdf', 'https://scs.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=0d70cc6d-acbc-4895-a4da-8932dc323723', 'hw/1/hw1.pdf', 'hw/1/handout.tar', 'jupyter.pdf', 'jupyter.tar', 'https://scs.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=08d57ec4-058f-476d-9962-356b3c39af37', 'relational_data.pdf', 'https://scs.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=d3f716a3-a874-415d-8b6b-5c0af4efd4c6', 'visualization.pdf', 'Visualization.ipynb', 'https://scs.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=c0e02818-c252-49a9-83b9-b5c52f01b12f', 'http://www.datasciencecourse.org/hw/2/handout.tar', 'matrices.pdf', 'NumpyBasics.ipynb', 'https://scs.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=70d8ac29-2865-4d71-a1f4-59127ec04c2c', 'tutorial.pdf', 'graphs.pdf', 'Netw